In [1]:
import torch

# Check if GPU is available≠
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU device
    current_device = torch.cuda.current_device()
    print(f"Current GPU device: {current_device}")

    # Get GPU properties
    gpu_properties = torch.cuda.get_device_properties(current_device)
    print(f"GPU Properties:\n{gpu_properties}")
else:
    print("No GPU available. Switching to CPU.")

Number of available GPUs: 1
GPU 0: Tesla P100-PCIE-16GB
Current GPU device: 0
GPU Properties:
_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16276MB, multi_processor_count=56)


In [ ]:
import torch
from torch import nn
from torchvision.transforms import ToTensor

# Task 1

## A

In [2]:
import torch
import torch.nn as nn

class OneAModule(nn.Module):
    def __init__(self, in_channels):
        super(OneAModule, self).__init__()

        # 1x1 conv branch
        self.branch1 = nn.Conv2d(in_channels, 128, kernel_size=1)

        # 1x1 conv followed by 3x3 conv branch
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 192, kernel_size=3, padding=1)
        )

        # 1x1 conv followed by 5x5 conv branch
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 96, kernel_size=5, padding=2)
        )

        # 3x3 max pooling followed by 1x1 conv branch
        self.branch4 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        # Concatenate the branches along the channel dimension
        output = torch.cat([branch1, branch2, branch3, branch4], dim=1)

        return output

# Example usage
in_channels = 256
model = OneAModule(in_channels)

# Print the model architecture
print(model)

# Example input tensor (replace with your data)
input_data = torch.randn(1, in_channels, 28, 28)  # Batch size of 1, 28x28x256 input

# Forward pass
output = model(input_data)
print("Output shape:", output.shape)


OneAModule(
  (branch1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (branch2): Sequential(
    (0): Conv2d(256, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (branch3): Sequential(
    (0): Conv2d(256, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (branch4): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
)
Output shape: torch.Size([1, 672, 28, 28])
